In [1]:
from utils.models import runfullseasonpredictor
from utils.mining_data_tb import getnbafromoddsportal
from utils.mining_data_tb import getnbafrommassey
from utils.sql_tb import nbacleantomysql
from utils.folders_tb import opennbacsv
from utils.folders_tb import openmodelresults
from utils.sql_tb import getpredictiontablefromsql
from utils.sql_tb import getfullnbatablefromsql
from utils.models import gamepredictor
from utils.models import runteamseasonpredictor
from utils.dataframes import showtotalresultsforateam

# <u> Collect data </u>

This project is obtained from two websites via webscraping, <u>https://masseyratings.com/nba/games </u> and <u>https://www.oddsportal.com/basketball/usa/nba/results/#/page/1 </u>.

Each game is sent to a json dictionary in the DATA folder and joined together with the relevant information from each page into a file called <u>nbaclean.csv</u> which is also sent to mySQL into the table named <u>daniel_walker</u>.

### **Beware, each may take up to a few hours to complete.**


In [ ]:
# BEWARE, THESE FUNCTIONS TAKE HOURS TO COMPLETE, to avoid unexpected errors the dictionaries made to jsons are set up to use new files instead of overwriting the existing ones

getnbafrommassey('https://masseyratings.com/nba/games?dt=20210601', maxdate=(datetime.today().strftime('%Y%m%d')))

getnbafromoddsportal(2)

In [2]:
nbacleantomysql("nbaclean.csv")

nbaclean.csv appended to daniel_walker table in database


'Done'

# <u> Game Prediction</u>

To make the first prediction, the program is given a team name, the game number that needs to be predicted, the regressor to be used and a variable.

The function will print the useful information regarding the game, the prediction, and also the result to see if the prediction was correct.


### **The function can also be called from the Streamlit app.**

In [2]:
Team = "Utah Jazz"
Gamenumber = 40
regressor = "DecisionTreeRegressor"
eon = "mse" #eon means ESTIMATOR (used in RandomForestRegressor, recommend 100), CRITERION (used in DecisionTreeRegressor, recommend mse) or
        #N_NEIGHBORS (used in KNeighborsRegressor, recommend 4)


gamepredictor(Team, Gamenumber, regressor= regressor, eon=eon)

Utah Jazz in game 40 vs Boston Celtics
-----------------------------
Bookies predicted points for Utah Jazz is: 114
Utah Jazz last 10 games scoring Avg is: 119.75
Over the previous 10 games Boston Celtics receiving Avg is: 116.1
In Boston Celtics previous 10 games, 8.0 of their games have gone over the bookies prediction
SYSTEM PREDICTED score:  119.5
Using the variable prediction, the programs recommendation is: Prediction is OVER 114
The SYSTEMS STRAIGHT PREDICTION is Over the bookies prediction
-----------------------------
Final Score (Real):  117
The SYSTEM STRAIGHT PREDICTION was:  Correct
-----------------------------


# <u> View a teams season predictions game by game </u>

Run "runteamseasonpredictor()" to return the DF detail for each game.

Run "showtotalresultsforateam()" to generate a CSV into DATA and return the DF with the teams season predictions and bet return results.

### **The function can also be called from the Streamlit app.**

In [4]:
Team = "New Orleans Pelicans"
regressor = "RandomForestRegressor"
eon = 100 #eon means ESTIMATOR (used in RandomForestRegressor, recommend 100), CRITERION (used in DecisionTreeRegressor, recommend mse) or
        #N_NEIGHBORS (used in KNeighborsRegressor, recommend 4)


In [ ]:
runteamseasonpredictor(Team, regressor, eon)

In [5]:
showtotalresultsforateam(Team, runteamseasonpredictor(Team, regressor, eon))

Appending done


,Regressor,EstimatorsorNeigbor,ContainsNotConclusivePred,Totalgames,GamesRemovedPred0,TotalCorrect,Correct,BetReturn,BetProfit,TotalOversPred,TotalOversCorrect,OversCorrect,OversBetReturn,OversProfit,TotalUndersPred,TotalUndersCorrect,UndersCorrect,UndersBetReturn,UndersProfit
0,RandomForestRegressor,100,True,62,1,38,61.29%,7.54,12.16%,39,24,61.54%,4.92,12.62%,23,14,60.87%,2.62,11.39%
1,RandomForestRegressor,100,False,16,0,13,81.25%,7.79,48.69%,7,6,85.71%,3.98,56.86%,9,7,77.78%,3.81,42.33%


# <u>Run function that takes a regressor or a list of regressors and then a variable to simulate the season and store the overall results in csv and mySQL </u>
The function simulates the whole season from game 10, game by game, team by team.
For each game the program trains and fits each model and makes a prediction using information from the previous 10 games.
The prediction made for each game is whether the predicted team score will be greater or lower than the bookies prediction.
If this prediction was correct, then that game would win the 1€ bet (returns 0.83€ as the odds are 1.83), if the prediction loses, then the 1€ bet is lost.
The teams results are stored in a dataframe to then be inserted into another function which totals up the columns and appends the overall results for the regressor and variable into a csv file and into a mySQL table named prediction.

For each game and each team <u>(1860 games)</u> a model is fitted with the scores available from the previous 10 games. This means that not every model is saved. The first model is saved in the MODELS folder.

## The following regressors can be used:
BayesianRidge

DecisionTreeRegressor

GaussianProcessRegressor

GradientBoostingRegressor

KNeighborsRegressor

LinearRegression

RadiusNeighborsRegressor

RandomForestRegressor

RidgeCV

SVR

### **This function can also be called from the Streamlit app.**

In [10]:
regressor = ["DecisionTreeRegressor"]
eon = "mse" #eon means ESTIMATOR (used in RandomForestRegressor, recommend 100), CRITERION (used in DecisionTreeRegressor, recommend mse) or
        #N_NEIGHBORS (used in KNeighborsRegressor, recommend 4)

runfullseasonpredictor(regressor, eon)

DecisionTreeRegressor
Model Pickle Saved
Model result appended to prediction table in database
Appending done


'Done'

# <u>View these DataFrames from either DATA csv files or from mySQL:</u>

Cleaned dataset without % formats and without unnecessary columns.

Overall Model Results

### **This function can also be called from the Streamlit app.**

In [ ]:
opennbacsv("nbaclean.csv")

In [ ]:
getfullnbatablefromsql()

In [ ]:
openmodelresults("modelresults.csv")

In [ ]:
getpredictiontablefromsql()